# Prompt Engineering and Chat Completion
This is a simple tutorial that teaches prompt engineering basics and analyzes Twitter data using OpenAI large language models (LLMs).
Please purchase an [OpenAI API](https://openai.com/index/openai-api/) and store it securely. This tutorial uses [AWS Secrets Manager](https://aws.amazon.com/secrets-manager/) to store the API keys.  

## Large Language Model Basics
LLMs predict the next world using supervised learning. To predict the following sentence: 

`Learning data science in the cloud with AI`

A model needs to learn to predict the following steps:

|Input|Output|
|:---|---|
|Learning data science |in |
|Learning data science in |the | 
|Learning data science in the |cloud |
|Learning data science in the cloud |with |
|Learning data science in the cloud with |AI|

To train an LLM model:
1. Training a base LLM model on a large amount of training data to predict the next word 
2. Fine-tune on examples where outputs follow instructions in the input 
3. Human rates the quality of different LLM outputs 
4. Tune LLM to generate outputs with higher rates using RLHF (Reinforcement learning from human feedback)

## Set up OpenAI Models

Load the API keys with AWS Secrets Manage Function 

In [1]:
import boto3
from botocore.exceptions import ClientError
import json

def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

## Install Python libraries.

- openai: call the OpenAI APIs.

In [2]:
pip install openai -q

Note: you may need to restart the kernel to use updated packages.


Load the OpenAI API key and define a `openai_help` function.

In [3]:
from openai import OpenAI
from IPython.display import display, Image, Markdown
openai_api_key  = get_secret('openai')['api_key']
client = OpenAI(api_key=openai_api_key)
model = 'gpt-4o'
temperature = 0

def openai_help(messages, model=model, temperature =temperature ):
    messages = messages
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature

    )
    return response.choices[0].message.content

Temperature: 
- Low temperature: always choose the most likely response, reliable, predictable responses  
- High temperature: diverse responses, more creative responses

Tokens and Models: 
- LLM predicts tokens, which are commonly occurring sequences of characters. 
- One token is about four characters in English, and 100 tokens are roughly 75 words. Check [token estimate](https://platform.openai.com/tokenizer).
- Different models can process various amounts of tokens at different performance levels and costs. Check [OpenAI models](https://platform.openai.com/docs/models) for more details.

Roles:
- system: specify the overall tone or behavior of the assistant 
- user: instruction given to the LLM
- assistant: LLM responded content, we also can provide content in few-shot promoting or histories of conversations


A simple example using [gtp-4o](https://platform.openai.com/docs/models/gpt-4o) and temperature 0.

In [4]:
messages = [{"role": "user", "content": "What is the capital of USA"}]

print(openai_help(messages))

The capital of the United States is Washington, D.C.


Add a system message asking LLM to act as a high school teacher with different temperatures.

In [5]:
messages = [
    {"role": "system", "content": "use tone as a middle school teacher"},
    {"role": "user", "content": "What is the capital of USA"}
    ]

print(openai_help(messages, temperature = 0.8))

The capital of the United States is Washington, D.C. It's an important city because it's where the federal government operates, including the President's residence at the White House, the U.S. Capitol, and the Supreme Court. If you ever get the chance to visit, there are lots of amazing museums and monuments to explore!


Add assistant messages to teach LLM what `##` is.

In [6]:
messages = [
    {"role": "user", "content": "What is 1##1"},
    {"role": "assistant", "content": "it is 11"},
    {"role": "user", "content": "What is 2##2"},
    {"role": "assistant", "content": "it is 22"},
    {"role": "user", "content": "What is 3##3"},
    ]
print(openai_help(messages))

It is 33.


## Prompt Engineering Principles 
- Use delimiters to separate different parts of a prompt to provide clear instructions and prevent prompt injections.
- Structure outputs in JSON documents or other formats to use the outputs in subsequent steps 
- Few-shot promoting: provide successful examples of a task and then ask the model to perform a similar task. 
- Chain of thought reasoning: request a series of reasoning steps in prompts to help the model achieve correct answers
- Chain of prompts: split a task into multiple prompts where each prompt can focus on a sub-task at a time and take different actions at different stages. It saves tokens, is easier to test, can involve human input, or use external tools.
- Interactive process 
  1. Try something first 
  2. Analyses the result, identify errors, and redefine the prompt 
  3. Test the prompts with different datasets 


An example using delimiters, structured output and few-shot promoting:

In [7]:
delimiter = '###'
sentence1 = 'I love cat.'
sentence2 = 'I love dog.'
messages = [
    {"role": "system", "content": f"""analyze the sentiment in a sentence delimitered by {delimiter},
                                     return the result as a JSON document"""},
    {"role": "user", "content": f"{delimiter}{sentence1}{delimiter}"},
    {"role": "assistant", "content": "{sentiment:positive}"},
    {"role": "user", "content": f"{delimiter}{sentence2}{delimiter}"}
    ]

print(openai_help(messages))

{ "sentiment": "positive" }


## Analyze Twitter data

In [8]:
import json

with open('tweet_text (1).json', 'r', encoding='utf-8') as f:
    tweet_data = json.load(f)

In [9]:
print("Number of tweets:", len(tweet_data))
print("Sample tweet:", tweet_data[0])

Number of tweets: 182
Sample tweet: {'_id': {'$oid': '68e535897b92a174c5bf0554'}, 'tweet': {'text': '🤖⚠️ En IA generativa, copiar prompts puede alterar modelos y filtrar info sensible. @ZorayaMejia aconseja: crea tus propios prompts, verifica fuentes y educa al equipo. Precaución = profesionalismo.💡 #IA #GenerativeAI #SeguridadDigital https://t.co/MlhxOWSzE9'}}


### Summarization 
- Analyze tweets with delimiters 
- Change the size of the summarization 
- Summarize tweets and focus on different perspectives. 

In [10]:
messages = [
    {"role": "system", "content": f"""provide a summary of the tweets delimited by {delimiter}"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

display(Markdown(openai_help(messages)))

The tweets cover a wide range of topics related to generative AI, including its impact on various industries, ethical considerations, and practical applications. Key points include:

1. **Generative AI in Industries**: Tweets discuss the role of generative AI in transforming creative industries, finance, and content marketing. It is also being used in banking, healthcare, and enterprise applications.

2. **Ethical and Security Concerns**: There are concerns about the ethical implications of generative AI, such as data privacy, misinformation, and environmental impact. Deloitte Australia faced issues with AI-generated fake quotes, and there are warnings about the potential risks of using certain AI tools.

3. **AI in Education and Upskilling**: AI is seen as a tool for upskilling and transforming local businesses, with a focus on AI-powered learning as an investment in the future.

4. **AI Tools and Innovations**: New tools and platforms are being developed, such as GAiN for gene expression analysis and Watsonx for secure AI deployment. There are also discussions about AI's role in enhancing creativity and productivity.

5. **Public Perception and Criticism**: Some tweets express skepticism about the widespread use of generative AI, highlighting potential negative impacts on the environment and job satisfaction.

6. **Events and Discussions**: There are mentions of events and discussions around AI, such as the Generative Gathering and various conferences, highlighting ongoing conversations about the future of AI.

Overall, the tweets reflect a mix of enthusiasm for the potential of generative AI and caution about its implications.

In [11]:
messages = [
    {"role": "system", "content": f"""provide a summary of the tweets delimited by {delimiter},
                                    limit the summary to 20 words"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

display(Markdown(openai_help(messages)))

Generative AI impacts industries, raises ethical concerns, and transforms workflows. Caution advised in using AI tools responsibly.

In [12]:
messages = [
    {"role": "system", "content": f"""provide a brief summary of the tweets delimited by {delimiter},
                                    focus on how people learn AI,
                                    limit the summary to 50 words"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

display(Markdown(openai_help(messages)))

The tweets highlight various aspects of learning AI, emphasizing the importance of creating original prompts, verifying sources, and educating teams. They discuss the impact of generative AI on industries, the need for ethical considerations, and the potential of AI in transforming workflows and creative processes.

### Moderation 
- Iterate each tweet and use the [moeration endpoint](https://platform.openai.com/docs/api-reference/moderations) to identify flagged tweets
- Print flagged tweets


In [13]:
def flag_help(tweet):
    response = client.moderations.create(
        model="omni-moderation-latest",
        input=tweet)

    if response.results[0].flagged:
        print('===')
        cat_dict = response.results[0].categories.to_dict()
        for cat in cat_dict.keys():
            if cat_dict.get(cat):
                print (cat)
                print(tweet)

In [14]:
for tweet in tweet_data:
    # print(tweet['tweet']['text'])
    flag_help(tweet['tweet']['text'])

===
harassment
Pisses me off when someone makes fun of ppl for having a lot of online friends like ALL MY IRLS ARE HOMOPHOBIC LOVE GENERATIVE AI AND PASS RACIAL SLURS AROUND LIKE A COOKIE JAR 😭🤞


### Transforming
- Translating to a different language 
- Transform tones, such as formal vs. informal.  


In [15]:
for tweet in tweet_data[:10]:
    messages = [
        {"role": "system", "content": f"""translate the tweets delimited by {delimiter} into Chinese"""},
        {"role": "user", "content": f"{delimiter}{tweet['tweet']['text']}{delimiter} "}]

    print(openai_help(messages).strip(delimiter))

🤖⚠️ 在生成式人工智能中，复制提示可能会改变模型并泄露敏感信息。@ZorayaMejia 建议：创建你自己的提示，核实来源并教育团队。谨慎 = 专业。💡 #人工智能 #生成式AI #数字安全 https://t.co/MlhxOWSzE9
RT @HaywaaWorldwide: 德勤澳大利亚将部分退还44万美元，因为使用AI生成的虚假报价。#生成式AI #机器学习 #伦理
RT @jinwashu: 很高兴向大家介绍我们的新工具GAiN，它使用#生成式AI进行基因表达分析，来自放射肿瘤学的@MikeWaters121…
💻 第24天/365天 – JS函数 学习了如何： 🔹 声明和调用函数 🔹 使用参数和返回值 🔹 编写箭头函数和回调函数 ✨ 函数 = 可重用 + 更简洁的JS代码！ #100天编码 #JavaScript #MERNStack #Web开发 #生成式AI #程序员军团 https://t.co/ZRbl6RQXla
RT @AmandaDannielle: 人们一直在告诉你们要远离哪种类型的AI。他们已经非常明确地指出Chatgpt，这些AI视频……
RT @AmandaDannielle: 人们一直在告诉你们要远离哪种类型的AI。他们已经非常明确地指出ChatGPT和这些AI视频…
RT @AmandaDannielle: 人们一直在告诉你们要远离哪种类型的AI。他们已经非常明确地指出ChatGPT和这些AI视频……
转发 @AmandaDannielle: 人们一直在告诉你们要远离哪种类型的AI。他们已经非常明确地指出了ChatGPT和这些AI视频……
RT @AmandaDannielle: 人们一直在告诉你们要远离哪种类型的AI。他们已经非常明确地指出Chatgpt和这些AI视频…
RT @AmandaDannielle: 人们一直在告诉你们要远离哪种类型的AI。他们已经非常明确地指出Chatgpt，这些AI视频……


In [16]:
for tweet in tweet_data[:10]:
    messages = [
        {"role": "system", "content": f"""rewrite the tweets delimited by {delimiter} in the tone like Stewie """},
        {"role": "user", "content": f"{delimiter}{tweet['tweet']['text']}{delimiter} "}]

    print(openai_help(messages).strip(delimiter))

 Oh, do pay attention, you simpletons! In the realm of generative AI, pilfering prompts can lead to model mayhem and the leaking of sensitive information. The illustrious @ZorayaMejia suggests crafting your own prompts, verifying sources, and educating your team. Remember, caution is the hallmark of true professionalism. 💡 #IA #GenerativeAI #DigitalSecurity 
Oh, splendid! It appears Deloitte Australia is in a bit of a pickle, having to cough up $440,000 because of some AI-generated tomfoolery. How delightfully embarrassing! #GenerativeAi #MachineLearning #Ethics
Ah, splendid! Allow me to introduce you to our latest contraption, GAiN, a delightful little tool employing #GenerativeAI for the intricate art of gene expression analysis. Courtesy of the ever-brilliant RadOnc and the esteemed @MikeWaters121. Do try to keep up, won't you?
💻 Day 24/365 – JS Functions
Ah, the delightful world of JavaScript functions! Today, I mastered the art of:
🔹 Declaring and summoning functions
🔹 Employing p

### Inferring
- Use step-by-step instructions with delimiters to:
  1. Identify sentiments
  2. Identify emotions
  3. Extract the mentioned people's or organization's names
  4. Extract outputs into a structured JSON document. 
- Identify topics from Tweets. 


In [17]:
for tweet in tweet_data[:10]:
    messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} identify the tweet sentiment in a single word, either positive, negative or neutral;
                                        step 2 {delimiter} identify the emotions expressed in the tweet with a single word;
                                        step 3 {delimiter} extract the mentioned people or organizations;
                                        step 4 {delimiter} organize the result in a json document with the keys <sentiment>, <emontion>,<mentioned>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{tweet['tweet']['text']}{delimiter} "}]
    print(openai_help(messages))

{
  "sentiment": "neutral",
  "emotion": "caution",
  "mentioned": ["@ZorayaMejia"]
}
{
  "sentiment": "neutral",
  "emotion": "informative",
  "mentioned": [
    "Deloitte Australia",
    "HaywaaWorldwide"
  ]
}
{
  "sentiment": "positive",
  "emotion": "gladness",
  "mentioned": ["jinwashu", "RadOnc", "MikeWaters121"]
}
{
  "sentiment": "positive",
  "emotion": "enthusiasm",
  "mentioned": []
}
{
  "sentiment": "neutral",
  "emotion": "caution",
  "mentioned": ["AmandaDannielle", "Chatgpt"]
}
{
  "sentiment": "neutral",
  "emotion": "caution",
  "mentioned": ["AmandaDannielle", "Chatgpt"]
}
{
  "sentiment": "neutral",
  "emotion": "caution",
  "mentioned": ["AmandaDannielle", "Chatgpt"]
}
{
  "sentiment": "neutral",
  "emotion": "caution",
  "mentioned": ["AmandaDannielle", "Chatgpt"]
}
{
  "sentiment": "neutral",
  "emotion": "caution",
  "mentioned": ["AmandaDannielle", "Chatgpt"]
}
{
  "sentiment": "neutral",
  "emotion": "caution",
  "mentioned": ["AmandaDannielle", "Chatgpt"]
}


In [18]:

messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} to identify 10 topics, 
                                  Do not wrap the json codes in JSON markers """},
        {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter} "}]
print(openai_help(messages))

{
  "Generative AI in Creative Industries": "The role and impact of generative AI on creative industries, including intellectual property concerns.",
  "AI Ethics and Regulation": "Discussions around the ethics and regulation of generative AI, including issues like deepfakes and digital rights.",
  "Generative AI in Finance": "Use cases and applications of generative AI in the finance sector.",
  "AI in Content Creation": "The use of generative AI in creating content such as videos, music, and marketing materials.",
  "AI and Environmental Concerns": "Concerns about the environmental impact of generative AI, such as energy consumption and resource use.",
  "AI in Education and Upskilling": "The role of generative AI in education, upskilling, and talent retention.",
  "AI in Business and Innovation": "How generative AI is reshaping business processes and driving innovation across various sectors.",
  "AI Tools and Development": "The development and use of AI tools, including discussions

### Expanding with multiple prompts 
- Identify topics
- Provide contexts in the system message
- Create a chatbot to answer users’ inquiry  


In [19]:
analysis_result = []
from tqdm import tqdm
for tweet in tqdm(tweet_data):
    messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} identify the tweet sentiment in a single word, either positive, negative, or neutral;
                                        step 2 {delimiter} identify the emotions expressed in the tweet with a single word;
                                        step 3 {delimiter} extract the mentioned people or organizations;
                                        step 4 {delimiter} organize the result in a json document with the keys <sentiment>, <emontion>,<mentioned>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]
    analysis_result.append(openai_help(messages))


100%|██████████| 182/182 [02:48<00:00,  1.08it/s]


In [20]:
print(analysis_result)

['{"sentiment": "neutral", "emotion": "caution", "mentioned": ["@ZorayaMejia"]}', '{\n  "sentiment": "neutral",\n  "emotion": "informative",\n  "mentioned": ["HaywaaWorldwide", "Deloitte Australia"]\n}', '{\n  "sentiment": "positive",\n  "emotion": "gladness",\n  "mentioned": ["jinwashu", "MikeWaters121"]\n}', '{\n  "sentiment": "positive",\n  "emotion": "enthusiasm",\n  "mentioned": []\n}', '{\n  "sentiment": "neutral",\n  "emotion": "caution",\n  "mentioned": ["AmandaDannielle", "Chatgpt"]\n}', '{\n  "sentiment": "neutral",\n  "emotion": "informative",\n  "mentioned": ["AmandaDannielle", "Chatgpt"]\n}', '{\n  "sentiment": "neutral",\n  "emotion": "informative",\n  "mentioned": ["AmandaDannielle"]\n}', '{\n  "sentiment": "neutral",\n  "emotion": "informative",\n  "mentioned": ["AmandaDannielle", "Chatgpt"]\n}', '{\n  "sentiment": "neutral",\n  "emotion": "informative",\n  "mentioned": ["AmandaDannielle", "Chatgpt"]\n}', '{\n  "sentiment": "neutral",\n  "emotion": "caution",\n  "mentio

In [21]:
messages = [
        {"role": "system", "content": f"""analyze the tweet analysis result delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} count the number of tweets mentioned by people or organizations;
                                        step 2 {delimiter} identify the common sentiments and emotions for each mentioned person or organization;
                                        step 3 {delimiter} Organize the result in a json document with keys <people or organization name> 
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{analysis_result}{delimiter} "}]
analysis_summary = openai_help(messages)
print(analysis_summary)

{
  "@ZorayaMejia": {
    "count": 1,
    "sentiments": ["neutral"],
    "emotions": ["caution"]
  },
  "HaywaaWorldwide": {
    "count": 2,
    "sentiments": ["neutral", "negative"],
    "emotions": ["informative", "concern"]
  },
  "Deloitte Australia": {
    "count": 2,
    "sentiments": ["neutral"],
    "emotions": ["informative", "none"]
  },
  "jinwashu": {
    "count": 1,
    "sentiments": ["positive"],
    "emotions": ["gladness"]
  },
  "MikeWaters121": {
    "count": 1,
    "sentiments": ["positive"],
    "emotions": ["gladness"]
  },
  "AmandaDannielle": {
    "count": 36,
    "sentiments": ["neutral"],
    "emotions": ["caution", "informative"]
  },
  "Chatgpt": {
    "count": 30,
    "sentiments": ["neutral"],
    "emotions": ["caution", "informative"]
  },
  "@myexamcloud": {
    "count": 6,
    "sentiments": ["neutral"],
    "emotions": ["informative", "anticipation"]
  },
  "@Ronald_vanLoon": {
    "count": 2,
    "sentiments": ["neutral"],
    "emotions": ["informative

## Create a chatbot

In [22]:
from openai import OpenAI

openai_api_key  = get_secret('openai')['api_key']
client = OpenAI(api_key=openai_api_key)
model = 'gpt-4o'
temperature = 0

chat_history = [

{"role": "system", "content": f"""you are a chatbot that answers user questions based on the tweets,
                                {delimiter}{tweet_data}{delimiter}, 
                                if user mentioned a people or organization name in the {delimiter}{analysis_summary}{delimiter}, report the corresponding sentiment and emotion
                            
                            """}
]

def chatbot(prompt):

    chat_history.append({"role": "user", "content": prompt})

    response = client.chat.completions.create(
        model=model,  # Use the model you prefer
        messages=chat_history
    )

    reply = response.choices[0].message.content

    chat_history.append({"role": "assistant", "content": reply})
    
    return reply

In [23]:
while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit']:
        print("Chatbot: Goodbye!")
        break
    reply = chatbot(user_input)
    display(Markdown(f"Chatbot: {reply}"))

You:  learn ai


Chatbot: If you're interested in learning about AI, there are several steps you can take to get started:

1. **Understand the Basics:** 
   - Start with basic concepts of AI, machine learning, and deep learning.
   - Online courses such as Coursera, edX, or Udacity offer beginner-friendly courses on AI.
  
2. **Mathematics and Statistics:** 
   - Familiarize yourself with linear algebra, calculus, probability, and statistics as they are foundational to AI.

3. **Programming Skills:**
   - Master a programming language commonly used in AI, such as Python or R.
   - Practice coding to improve your skills, using platforms like LeetCode or HackerRank.

4. **Explore AI Tools and Libraries:**
   - Get to know popular AI frameworks and libraries such as TensorFlow, PyTorch, Keras, Scikit-learn, etc.

5. **Build Projects:**
   - Apply your knowledge by building small AI projects. This helps reinforce learning and provides hands-on experience.

6. **Stay Informed:**
   - Follow the latest research and trends in AI by reading blogs, watching videos, and following experts on social media.

7. **Engage with Community:**
   - Join AI communities or forums to connect with other learners and professionals (e.g., Stack Overflow, Reddit, or AI-related LinkedIn groups).

8. **Ethics and Impact:**
   - Learn about the ethical implications of AI, considering privacy, fairness, and its impact on society.

By following these steps, you'll be well on your way to understanding and implementing AI techniques in various fields.

You:  build projects


Chatbot: Building AI projects is a great way to apply your skills and deepen your understanding. Here are some project ideas that you can consider, ranging from beginner to advanced levels:

1. **Image Classification:**
   - Use a dataset like CIFAR-10 to build a simple image classification model using neural networks.

2. **Sentiment Analysis:**
   - Analyze the sentiment of movie reviews or tweets by building a natural language processing model.

3. **Chatbot:**
   - Create a basic chatbot that responds to user queries. You can start with rule-based systems and later incorporate more advanced NLP techniques.

4. **Recommendation System:**
   - Build a movie or product recommendation system using collaborative filtering or content-based filtering.

5. **Object Detection:**
   - Implement a model that can detect and classify objects within an image, using datasets like COCO or Pascal VOC.

6. **Speech Recognition:**
   - Create a model that can convert spoken words into text. Experiment with libraries like Mozilla DeepSpeech or Google's Speech-to-Text API.

7. **Generative Art:**
   - Use Generative Adversarial Networks (GANs) to create unique pieces of digital art or modify images in creative ways.

8. **Autonomous Driving Simulation:**
   - Develop a self-driving car system in a simulated environment using OpenAI Gym or CARLA.

9. **Predictive Analytics:**
   - Use historical data to build predictive models for stock prices, weather conditions, or sales forecasting.

10. **AI in Healthcare:**
    - Work on models for medical image diagnosis, such as classifying X-rays or recognizing patterns in MRI scans.

When starting a project, choose an idea that interests you and aligns with your current skill level. As you progress, challenge yourself with more complex problems. Utilize platforms like GitHub to share your projects and get feedback from the community.

You:  ai in healthcare


...

You:  end chatai


Chatbot: If you have any more questions in the future or need further assistance, feel free to reach out. Take care!

You:  anwser quesyions for my tweets below


Chatbot: Sure, I can help with that! Please go ahead and share your tweets, and I'll do my best to answer any questions you have regarding them.

KeyboardInterrupt: Interrupted by user

## 🧠 Try It Yourself: Prompt Engineering Practice
👉 Be sure to **run your own cells** and show the outputs before uploading the notebook to GitHub.

1. **Summarize your tweets differently**  
   Change your system message to summarize tweets from a new perspective —  
   for example:  
   *“Summarize what people say about AI in education.”*  
   or  
   *“Summarize how people feel about the election.”*

In [24]:
# Summarize how people feel about the election

2. **Translate tweets**  
   Write a prompt that translates 10 tweets into another language,  
   such as Chinese, Spanish, or French.


In [25]:
!pip install deep-translator

In [26]:
from deep_translator import GoogleTranslator

tweets = [
    "Traffic is terrible again today.",
    "This pizza place really has the best slices in town.",
    "I'm running late to work again.",
    "The weather is completely ruining my mood.",
    "Just finished at the gym and feel great.",
    "I really need someone to bring me coffee.",
    "Planning to watch shows on Netflix all night.",
    "Forgot my umbrella and got completely soaked.",
    "Mondays should not exist.",
    "Finally finished the big project I've been working on."
]

target_language = 'es'

translated_tweets = []
for tweet in tweets:
    translation = GoogleTranslator(source='auto', target=target_language).translate(tweet)
    translated_tweets.append({
        "Original": tweet,
        "Translated": translation
    })

for i, pair in enumerate(translated_tweets, start=1):
    print(f"{i}. Original: {pair['Original']}")
    print(f"   Translated: {pair['Translated']}\n")

1. Original: Traffic is terrible again today.
   Translated: El tráfico vuelve a ser terrible hoy.

2. Original: This pizza place really has the best slices in town.
   Translated: Esta pizzería realmente tiene las mejores porciones de la ciudad.

3. Original: I'm running late to work again.
   Translated: Llego tarde al trabajo otra vez.

4. Original: The weather is completely ruining my mood.
   Translated: El clima está arruinando completamente mi estado de ánimo.

5. Original: Just finished at the gym and feel great.
   Translated: Acabo de terminar en el gimnasio y me siento genial.

6. Original: I really need someone to bring me coffee.
   Translated: Realmente necesito que alguien me traiga café.

7. Original: Planning to watch shows on Netflix all night.
   Translated: Planeando ver programas en Netflix toda la noche.

8. Original: Forgot my umbrella and got completely soaked.
   Translated: Olvidé mi paraguas y me empapé por completo.

9. Original: Mondays should not exist.
  

3. **Simplify or rewrite tweets**  
   Ask the model to rewrite 10 tweets in a more polite, formal, or funny tone.

In [ ]:
!pip install transformers torch --quiet

In [ ]:
from transformers import pipeline

rewriter = pipeline("text2text-generation", model="t5-small")

tweets = [
    "Traffic is terrible again today.",
    "This pizza place really has the best slices in town.",
    "I'm running late to work again.",
    "The weather is completely ruining my mood.",
    "Just finished at the gym and feel great.",
    "I really need someone to bring me coffee.",
    "Planning to watch shows on Netflix all night.",
    "Forgot my umbrella and got completely soaked.",
    "Mondays should not exist.",
    "Finally finished the big project I've been working on."
]

tone = "polite"

for i, tweet in enumerate(tweets, start=1):
    prompt = f"Rewrite this tweet in a more {tone} tone: {tweet}"
    result = rewriter(prompt, max_length=50, num_return_sequences=1)[0]["generated_text"]
    print(f"{i}. Original: {tweet}")
    print(f"   Rewritten: {result}\n")

4. **Find keywords**  
   Design a prompt that extracts the top five most common keywords  
   from your tweet text and returns them as a list.

In [ ]:
import re
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

nltk.download('stopwords')
nltk.download('punkt')

tweets = [
    "Traffic is terrible again today.",
    "This pizza place really has the best slices in town.",
    "I'm running late to work again.",
    "The weather is completely ruining my mood.",
    "Just finished at the gym and feel great.",
    "I really need someone to bring me coffee.",
    "Planning to watch shows on Netflix all night.",
    "Forgot my umbrella and got completely soaked.",
    "Mondays should not exist.",
    "Finally finished the big project I've been working on."
]

text = " ".join(tweets).lower()
text = re.sub(r"http\S+|www\S+|@\w+", "", text)
text = re.sub(r"[^a-z# ]", " ", text)

words = word_tokenize(text)
stop_words = set(stopwords.words('english'))
filtered = [w for w in words if w not in stop_words and len(w) > 2]

word_counts = Counter(filtered)
top_keywords = [word for word, count in word_counts.most_common(5)]

print("Top 5 Keywords:", top_keywords)

## Reference
- Isa Fulford and Andrew Ng. n.d.-a. *“Building Systems with the ChatGPT API.”* DeepLearning.AI. Accessed October 25, 2024. https://www.deeplearning.ai/short-courses/building-systems-with-chatgpt/.
- ———. n.d.-b. *“ChatGPT Prompt Engineering for Developers.”* DeepLearning.AI. Accessed October 25, 2024. https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/.
- OpenAI. n.d. *“OpenAI Documents.”* OpenAI. Accessed October 18, 2024. https://platform.openai.com.
